# **Koneksi Ke Google Big Query**

In [1]:
from google.colab import auth
from google.colab import data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
import numpy as np

# PRoses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
from google.cloud import bigquery

# Buat BigQuery client
project_id = 'impactful-drive-468912-m4'
bq_client = bigquery.Client(project = project_id)

## **Proses Ekstraksi Data dari sumber (Extract)**

In [3]:
%%bigquery data_transaksi --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_transaction

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
%%bigquery data_product --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_product

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
%%bigquery data_funnel --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_funnel

Query is running:   0%|          |

Downloading:   0%|          |

Setelah query dijalankan maka akan disimpan sebagai dataframe pandas

## **Periksa Informasi Umum pada Data**

In [4]:
# Periksa info umum pada data transaksi
data_transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829262 entries, 0 to 829261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   trx_id      829257 non-null  object
 1   product_id  829262 non-null  object
 2   trx_date    829262 non-null  Int64 
 3   trx_time    829262 non-null  dbtime
 4   units       829251 non-null  Int64 
dtypes: Int64(2), dbtime(1), object(2)
memory usage: 33.2+ MB


In [7]:
# Periksa info umum pada data product
data_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        37 non-null     object
 1   product_name      37 non-null     object
 2   product_category  37 non-null     object
 3   product_cost      37 non-null     object
 4   product_price     37 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [8]:
# Periksa info umum pada data funnel
data_funnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15950 entries, 0 to 15949
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         15950 non-null  Int64 
 1   product_id   15950 non-null  object
 2   purchase     15950 non-null  Int64 
 3   add_to_cart  15950 non-null  Int64 
 4   click        15950 non-null  Int64 
 5   view         15950 non-null  Int64 
dtypes: Int64(5), object(1)
memory usage: 825.7+ KB


## **Proses Transformasi Data (Transform)**

In [15]:
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
697611,DQTrx593661,DQProduk-003,28042025,00:51:29,2
4036,DQTrx044889,DQProduk-001,15022024,10:16:53,1
322820,DQTrx626986,DQProduk-013,17052025,09:12:36,1
269439,DQTrx704010,DQProduk-009,04072025,14:18:59,1
78925,DQTrx375711,DQProduk-003,02122024,18:23:38,1
586652,DQTrx602468,DQProduk-027,01052025,16:06:42,1
214447,DQTrx251802,DQProduk-008,19082024,09:21:26,1
78189,DQTrx363870,DQProduk-003,24112024,18:49:50,1
59673,DQTrx461837,DQProduk-002,30012025,21:15:07,1
604596,DQTrx138405,DQProduk-030,12052024,07:48:17,1


In [20]:
# Konversi kolom trx_date dari Integer menjadi string
data_transaksi['trx_date'] = data_transaksi['trx_date'].astype('str')

# Buat kondisi
data_transaksi['trx_date'] = np.where(data_transaksi['trx_date'].str.len() == 7, '0' + data_transaksi['trx_date'], data_transaksi['trx_date'])

data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
76335,DQTrx337957,DQProduk-003,04112024,14:53:23,1
531500,DQTrx205798,DQProduk-025,08072024,07:53:44,1
226537,DQTrx417914,DQProduk-008,30122024,08:22:55,1
40196,DQTrx741345,DQProduk-001,28072025,02:23:37,1
379107,DQTrx064940,DQProduk-017,07032024,04:23:33,1
419317,DQTrx429181,DQProduk-018,06012025,18:41:17,1
680370,DQTrx206648,DQProduk-034,08072024,06:36:49,1
286754,DQTrx393572,DQProduk-010,15122024,06:57:13,1
352286,DQTrx260699,DQProduk-015,27082024,11:25:47,1
445153,DQTrx480526,DQProduk-019,12022025,21:29:20,1


In [22]:
data_transaksi['trx_date'] = pd.to_datetime(data_transaksi['trx_date'], format = '%d%m%Y')
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
172510,DQTrx056470,DQProduk-007,2024-02-27,09:54:36,1
698361,DQTrx682387,DQProduk-003,2025-06-21,11:08:57,2
519271,DQTrx786221,DQProduk-024,2025-08-31,07:44:20,1
104473,DQTrx803155,DQProduk-003,2025-09-13,13:28:12,1
779805,DQTrx099132,DQProduk-017,2024-04-09,10:02:25,3
14366,DQTrx173991,DQProduk-001,2024-06-10,16:08:36,1
345797,DQTrx825910,DQProduk-014,2025-09-29,23:52:25,1
666128,DQTrx126648,DQProduk-032,2024-05-01,23:25:45,1
608471,DQTrx243414,DQProduk-030,2024-08-11,06:16:08,1
298843,DQTrx445885,DQProduk-011,2025-01-19,01:07:28,1


In [24]:
# Data asli (sebelum di drop)
data_transaksi.shape

(829262, 5)

In [25]:
# Hapus data yang NULL
data_transaksi = data_transaksi[data_transaksi['trx_id'].notnull()]
data_transaksi.shape

(829257, 5)

In [27]:
data_transaksi['units'] = data_transaksi['units'].fillna(0)

/tmp/ipython-input-1775575313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_transaksi['units'] = data_transaksi['units'].fillna(0)


In [30]:
data_transaksi[data_transaksi['units'] == 0]

,trx_id,product_id,trx_date,trx_time,units
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,0
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,0
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,0
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,0
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,0
5,DQTrx829213,DQProduk-019,2025-09-30,16:06:16,0
6,DQTrx829235,DQProduk-019,2025-09-30,17:49:03,0
7,DQTrx829167,DQProduk-024,2025-09-30,21:51:24,0
8,DQTrx829168,DQProduk-025,2025-09-30,02:05:10,0
9,DQTrx829254,DQProduk-025,2025-09-30,08:12:00,0


In [34]:
data_transaksi.duplicated(subset = 'trx_id').sum()

np.int64(0)

In [35]:
data_transaksi.duplicated().sum()

np.int64(0)

In [32]:
data_product.head(10)

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,IDR 149850,IDR 164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,IDR 29850,IDR 59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,IDR 29850,IDR 44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,IDR 44850,IDR 149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,IDR 59850,IDR 164850
5,DQProduk-004,Notebook A5 DQLab,Buku Catatan,IDR 149850,IDR 194850
6,DQProduk-006,Earphone Kantor DQLab,Elektronik,IDR 104850,IDR 224850
7,DQProduk-021,Kalkulator Besar DQLab,Elektronik,IDR 104850,IDR 149850
8,DQProduk-032,Timbangan Mini DQLab,Elektronik,IDR 179850,IDR 224850
9,DQProduk-013,Headset Kantor DQLab,Elektronik,IDR 224850,IDR 314850


In [33]:
for col in ['product_cost','product_price']:
  data_product[col] = data_product[col].str.replace('IDR','')
  data_product[col] = data_product[col].astype(pd.Int64Dtype())

data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,149850,164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,29850,59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,29850,44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,44850,149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,59850,164850


In [36]:
data_product.duplicated().sum()

np.int64(2)

In [37]:
data_product[data_product.duplicated(subset = 'product_id', keep = False)]

,product_id,product_name,product_category,product_cost,product_price
17,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
18,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
27,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850
28,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850


In [38]:
data_product.shape

(37, 5)

In [39]:
data_product = data_product.drop_duplicates(subset = ['product_id'], keep = 'first')
data_product.shape

(35, 5)

In [40]:
data_integration = data_transaksi.merge(data_product, on = 'product_id' , how = 'left')
data_integration.head()

,trx_id,product_id,trx_date,trx_time,units,product_name,product_category,product_cost,product_price
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,0,Set Pulpen DQLab,Alat Tulis,44850,149850
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,0,Set Tulis Eksekutif DQLab,Merchandise,209850,299850
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,0,Printer Mini DQLab,Elektronik,524850,599850
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,0,Kertas Warna DQLab,Kertas & Cetak,209850,239850


## **Proses Simpan Data (Load)**

In [41]:
import pandas as pd

# Upload ke BigQuery - tabel belum ada
to_gbq(
    data_integration,
    destination_table = 'tbl_dwh_dqcommerce.tbl_integration',
    project_id = project_id,
    if_exists = 'fail'
)

100%|██████████| 1/1 [00:00<00:00, 2043.01it/s]


---

### Overview Total Penjualan Periode Data Tiap Bulan

In [ ]:
data_transaksi['month_year'] = data_transaksi['trx_date'].dt.strftime('%Y-%m')
data_transaksi.head()

,trx_id,product_id,trx_date,trx_time,units,month_year
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,<NA>,2025-09
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,<NA>,2025-09
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,<NA>,2025-09
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,<NA>,2025-09
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,<NA>,2025-09


In [ ]:
penjualan_bulanan = data_transaksi.groupby(['month_year'], as_index = False).agg(total = ('units', 'sum'))
penjualan_bulanan.head()

,month_year,total
0,2024-01,38009
1,2024-02,36935
2,2024-03,39981
3,2024-04,47102
4,2024-05,46910


In [ ]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_bulanan,
    x="month_year",
    y="total",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce</b>",
    color_discrete_sequence = ['#39074E'],
    markers=False)

"""
fig.add_vrect(
    x0="2025-03", x1="2025-09",
    fillcolor="#E73c35",
    opacity=0.5,
    layer="below",
    line_width=0,
    #annotation_text="Tahun 2025",
    #annotation_position="top "
)
"""

fig.update_layout(
    height = 450,
    width = 1100,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [ ]:
penjualan_per_produk = data_transaksi.groupby(['month_year', 'product_id'], as_index = False).agg(total = ('units', 'sum'))
penjualan_per_produk.head()

,month_year,product_id,total
0,2024-01,DQProduk-001,3441
1,2024-01,DQProduk-002,1513
2,2024-01,DQProduk-003,76
3,2024-01,DQProduk-004,166
4,2024-01,DQProduk-005,276


In [ ]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_per_produk, #[penjualan['month_year'] >= '2025-01'],
    x="month_year",
    y="total",
    color="product_id",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    markers=True
)

fig.update_layout(
    height = 550,
    width = 1200,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [ ]:
import plotly.graph_objects as go

highlight_product = ['DQProduk-003', 'DQProduk-019', 'DQProduk-025', 'DQProduk-018', 'DQProduk-031']

fig = go.Figure()

# Loop setiap produk
for product in penjualan_per_produk['product_id'].unique():
    df_product = penjualan_per_produk[penjualan_per_produk['product_id'] == product]

    # Tentukan properti styling
    if product in highlight_product:
        line_color = 'purple'
        line_width = 3
    else:
        line_color = 'lightgray'
        line_width = 1.2

    # Tambahkan trace
    fig.add_trace(go.Scatter(
        x=df_product['month_year'],
        y=df_product['total'],
        mode='lines+markers',
        name=product,
        line=dict(
            width=line_width,
            color=line_color
        )
    ))

# Layout styling
fig.update_layout(
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    height=550,
    width=1200,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        title='',
        showgrid=False,
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        showgrid=False,
    )
)

fig.show()